In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path_train = '/kaggle/input/mlda-hw4/Train.csv'
path_test = '/kaggle/input/mlda-hw4/Test.csv'
train_data = pd.read_csv(path_train)
test_data = pd.read_csv(path_test)

datetime_train = pd.DatetimeIndex(train_data['date'])
datetime_test = pd.DatetimeIndex(test_data['date'])

train_data['date'] = (datetime_train.year - 2011) * 12 + datetime_train.month
train_data = train_data.fillna(train_data.mean())

test_data['date'] = (datetime_test.year - 2011) * 12 + datetime_test.month
test_data = test_data.fillna(test_data.mean())

train_data.head()



In [ ]:
train_data.info()

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

cat_feat = ('street_id', 'balcon', 'rooms')
num_feat = ('date', 'g_lift', 'build_tech',
            'floor', 'area', 'metro_dist',
                    'kw1', 'kw2', 'kw3',
                    'kw4', 'kw5', 'kw6',
                    'kw7', 'kw8', 'kw9',
                    'kw10', 'kw11', 'kw12',
                    'kw13')

pipe = Pipeline([
    ('data_process', ColumnTransformer([
        ('numeric', 'passthrough', num_feat),
        ('cat', OneHotEncoder(), cat_feat)
    ])
    ),
    ('boosting', GradientBoostingRegressor(n_estimators=2000,
                                           learning_rate=0.9,
                                           loss='huber',
                                          verbose=1
                                          )
    )
    
])

In [ ]:
pipe

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X = train_data.drop(columns=['price'])
y = train_data['price']

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=0,
                                                 test_size=0.3)

# reg = GradientBoostingRegressor(n_estimators=2000, learning_rate=0.9,
#                                loss='huber', max_depth=8, verbose=1)

pipe.fit(X_train, y_train)

In [ ]:
pred = pipe.predict(X_val)

In [ ]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(y_val, pred)

TEST_DATA

In [ ]:
# from sklearn. preprocessing import OneHotEncoder

# cat_features = ['date','build_tech', 'floor', 'area', 'rooms', 'balcon',
#                'street_id', 'g_lift', 'n_photos']

# encoder = OneHotEncoder(handle_unknown='ignore')

# for feature in cat_features:

#     encoder_df = pd.DataFrame(encoder.fit_transform(test_data[[feature]]).toarray(),
#                              columns=encoder.get_feature_names_out())

#     test_data = test_data.join(encoder_df)

#     test_data.drop(feature, axis= 1 , inplace= True )

# test_data

In [ ]:
preds = pipe.predict(test_data)

idx = []
pred = []
for i in range(len(preds)):
    idx.append(100000 + i)
    pred.append(preds[i])

result = pd.DataFrame({'id': idx, 'price': pred})
result.to_csv('with_pipeline_225.csv', index=False)